In [66]:
from autoencoders import Conv1dAutoEncoder, LSTMAutoEncoder, TickerDataModule, MLPAutoEncoder

import pandas as pd
import numpy as np
from math import floor
import torch

from tqdm.notebook import tqdm

In [4]:
model_mlp = MLPAutoEncoder.load_from_checkpoint('autoencoders/lightning_logs/version_3/checkpoints/epoch=149-step=105599.ckpt',
                                            in_features=30,
                                            latent_features=8)
model_lstm = LSTMAutoEncoder.load_from_checkpoint('autoencoders/lightning_logs/version_1/checkpoints/epoch=29-step=99329.ckpt',
                                            seq_len=100,
                                            n_features=1)
model_cae = Conv1dAutoEncoder.load_from_checkpoint('autoencoders/lightning_logs/version_2/checkpoints/epoch=99-step=70399.ckpt',
                                            in_channels=1,
                                            n_latent_features=1)
model_mlp.eval()
model_lstm.eval()
model_cae.eval();

1


C:\Users\artem\anaconda3\envs\ml\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:
df = pd.read_csv('data/ticker_data_Close.csv', index_col=[0]).dropna(axis=1).pct_change().fillna(0).T

In [58]:
test_30 = df.values[100, 100:190]
test_30 = torch.tensor(test_30.reshape(3, 1, 30)).float()

In [60]:
test = model_cae.predict_step(test_30)
test.squeeze_()
test.shape

torch.Size([3, 18])

In [61]:
test = test.detach().numpy()

In [62]:
test.mean(axis=0)

array([ 1.3172268 , -0.20717724,  1.2663149 ,  1.2335238 ,  0.87040025,
        2.497875  ,  1.4231535 , -0.36850038, -3.012929  ,  0.344663  ,
        0.3438032 ,  0.28385302,  0.33017513, -0.5936204 ,  0.30444106,
       -1.093338  , -1.1015083 , -0.6877598 ], dtype=float32)

In [49]:
torch.nn.MSELoss()(test_30, model_mlp(test_30))

C:\Users\artem\anaconda3\envs\ml\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([3, 30])) that is different to the input size (torch.Size([3, 1, 30])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0005, grad_fn=<MseLossBackward0>)

In [72]:
mlp_encoded = np.zeros((df.shape[0], 8))
lstm_encoded = np.zeros((df.shape[0], 16))
cae_encoded = np.zeros((df.shape[0], 18))

for i, name_ticker in tqdm(enumerate(np.unique(df.index))):
    ts_name = df[df.index == name_ticker].values
    ts_name = ts_name.flatten()
    seq_len = ts_name.shape[0]
    fl_1 = floor(seq_len / 30)
    sample_1 = ts_name[:30 * fl_1].reshape(fl_1, 1, 30)
    fl_2 = floor(seq_len / 100)
    sample_2 = ts_name[:100 * fl_2].reshape(fl_2, 1, 100)
    
    mlp_sample = model_mlp.predict_step(torch.tensor(sample_1).float()).detach().numpy()
    cae_sample = model_cae.predict_step(torch.tensor(sample_1).float()).squeeze().detach().numpy()
    # lstm_sample = model_lstm.predict_step(torch.tensor(sample_2).float()).detach().numpy()
    
    mlp_emb = mlp_sample.mean(axis=0)
    cae_emb = cae_sample.mean(axis=0)
    # lstm_emb = lstm_sample.mean(axis=0)
    
    mlp_encoded[i, :] = mlp_emb
    cae_encoded[i, :] = cae_emb
    # lstm_encoded[i, :] = lstm_emb
        

0it [00:00, ?it/s]

In [79]:
df_mlp = pd.DataFrame(mlp_encoded, index=np.unique(df.index))
df_cae = pd.DataFrame(cae_encoded, index=np.unique(df.index))

In [81]:
df_mlp.to_csv('data/nn_data/ticker_data_mlp.csv')
df_cae.to_csv('data/nn_data/ticker_data_cae.csv')